<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/Get_GR2Analyst_Soundings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="+4" color="green"><b>Get GR2Analyst Soundings</b></font>  
<font size="+1" color="gray"><i>updated January 3, 2025</i></font>  
<font color="gray"><i>version 0.22</i></font>  
<hr>  

Starting with GR2Analyst Version 3.3.0.3, it is possible to copy in JSON formatted environmental soundings. This is important when testing [User Defined Products](https://www.grlevelx.com/downloads/UserDefinedProducts_3.pdf) (UDPs) that leverage isothermal levels and layers.  
<br />
The JSON formatting syntax can be seen with this real-time sounding here:

- https://www.grlevelx.com/soundings/getlatest.php?lat=34&lon=-85


<br />

<p>For historical data, the user can choose from the following:</p>  

<ul>
  <li>
    Model Soundings from Iowa State API. Example:
    <ul>
      <li>
        <a href="https://mesonet.agron.iastate.edu/api/1/nws/bufkit.json?time=2024-05-07T22:00:00&lon=-85.54&lat=42.89&gr=1">https://mesonet.agron.iastate.edu/api/1/nws/bufkit.json?time=2024-05-07T22:00:00&lon=-85.54&lat=42.89&gr=1</a>
      </li>
    </ul>
  </li>
  <li>
    Observed raobs from the U of WY website. Example:
    <ul>
      <li>
        <a href="https://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2024&MONTH=05&FROM=0800&TO=0800&STNM=72632">https://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2024&MONTH=05&FROM=0800&TO=0800&STNM=72632</a>
      </li>
    </ul>
  </li>

</ul>


<br/>

GR2Analyst automatically grabs historical model sounding data from the Iowa State API. You may want to overwrite this, which is possible if you change the times accordingly.  For example, by changing the "time" value below:

```
{
 "time" : "2025-01-02T21:00:00Z",
 "lat"  : 34.0111,
 "lon"  : -85.1150,
 "source" :
  {
   "type"          : "model",
   "model"         : "RAP",
   "row"           : 96,
   "col"           : 237,
   "run_time"      : "2025-01-02T20:00:00Z",
   "forecast_hour" : 1
  },
```

to one minute after the hour, like this ...
```
{
 "time" : "2025-01-02T21:01:00Z",
 ... },
```
   
it will override the sounding that automatically gets retrieved. GR2Analyst environmental soundings are stored as raob files in this Windows directory:
```
C:\Users\{your username}\AppData\Roaming\GRLevelX\GR2Analyst_3\Soundings
```
<br/>
These files can be deleted if particular hours get too cluttered.

<br/> <br/>

---
Questions or bugs? Please contact me at thomas.turnage@noaa.gov

---




In [ ]:
# @title <font size="+3" color="green">Run cell below to set up configuration</font>
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings("ignore")

TIME_REGEX = "[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}Z"

radars = {'PABC': (60.79, -161.88), 'PACG': (56.85, -135.53), 'PAHG': (60.73, -151.35),
          'PAKC': (58.68, -156.63), 'PAIH': (59.46, -146.3), 'PAEC': (64.51, -165.3),
          'PAPD': (65.04, -147.5), 'KBMX': (33.17, -86.77), 'KEOX': (31.46, -85.46),
          'KHTX': (34.93, -86.08), 'KMXX': (32.54, -85.79), 'KMOB': (30.68, -88.24),
          'KSRX': (35.29, -94.36), 'KLZK': (34.84, -92.26), 'KFSX': (34.57, -111.2),
          'KIWA': (33.29, -111.67), 'TPHX': (33.42, -112.16), 'KEMX': (31.89, -110.63),
          'KYUX': (32.5, -114.66), 'KBBX': (39.5, -121.63), 'KEYX': (35.1, -117.56),
          'KBHX': (40.5, -124.29), 'KVTX': (34.41, -119.18), 'KDAX': (38.5, -121.68),
          'KNKX': (32.92, -117.04), 'KMUX': (37.16, -121.9), 'KHNX': (36.31, -119.63),
          'KSOX': (33.82, -117.64), 'KVBX': (34.84, -120.4), 'KFTG': (39.79, -104.55),
          'KGJX': (39.06, -108.21), 'KPUX': (38.46, -104.18), 'TDEN': (39.73, -104.53),
          'KDOX': (38.83, -75.44), 'KEVX': (30.57, -85.92), 'KJAX': (30.48, -81.7),
          'KBYX': (24.6, -81.7), 'KMLB': (28.11, -80.65), 'KAMX': (25.61, -80.41),
          'TFLL': (26.14, -80.34), 'TMIA': (25.76, -80.49), 'TMCO': (28.34, -81.33),
          'TTPA': (27.86, -82.52), 'TPBI': (26.69, -80.27), 'KTLH': (30.4, -84.33),
          'KTBW': (27.71, -82.4), 'KFFC': (33.36, -84.57), 'KVAX': (30.89, -83.0),
          'KJGX': (32.68, -83.35), 'TATL': (33.65, -84.26), 'PGUA': (13.46, 144.81),
          'PHKI': (21.89, -159.55), 'PHKM': (20.13, -155.78), 'PHMO': (21.13, -157.18),
          'PHWA': (19.09, -155.57), 'KDMX': (41.73, -93.72), 'KDVN': (41.61, -90.58),
          'KCBX': (43.49, -116.24), 'KSFX': (43.11, -112.69), 'KLOT': (41.6, -88.08),
          'KILX': (40.15, -89.34), 'TMDW': (41.65, -87.73), 'TORD': (41.8, -87.86),
          'KVWX': (38.26, -87.72), 'KIWX': (41.36, -85.7), 'KIND': (39.71, -86.28),
          'TIDS': (39.64, -86.44), 'RODN': (26.31, 127.9), 'RKSG': (37.21, 127.29),
          'RKJK': (35.92, 126.62), 'KDDC': (37.76, -99.97), 'KGLD': (39.37, -101.7),
          'TICH': (37.51, -97.44), 'KTWX': (39.0, -96.23), 'KICT': (37.65, -97.44),
          'KHPX': (36.74, -87.29), 'KJKL': (37.59, -83.31), 'KLVX': (37.98, -85.94),
          'KPAH': (37.07, -88.77), 'TSDF': (38.05, -85.61), 'KPOE': (31.16, -92.98),
          'KLCH': (30.13, -93.22), 'KLIX': (30.34, -89.83), 'KHDC': (30.52, -90.41),
          'KSHV': (32.45, -93.84), 'TMSY': (30.02, -90.4), 'KBOX': (41.96, -71.14),
          'TBOS': (42.16, -70.93), 'TADW': (38.7, -76.84), 'TBWI': (39.09, -76.63),
          'TDCA': (38.76, -76.96), 'KCBW': (46.04, -67.81), 'KGYX': (43.89, -70.26),
          'KDTX': (42.7, -83.47), 'KAPX': (44.91, -84.72), 'KGRR': (42.89, -85.54),
          'KMQT': (46.53, -87.55), 'TDTW': (42.11, -83.51), 'KDLH': (46.84, -92.21),
          'KMPX': (44.85, -93.57), 'TMSP': (44.87, -92.93), 'KEAX': (38.81, -94.26),
          'KSGF': (37.24, -93.4), 'KLSX': (38.7, -90.68), 'TMCI': (39.5, -94.74),
          'TSTL': (38.81, -90.49), 'KGWX': (33.9, -88.33), 'KDGX': (32.28, -89.98),
          'KBLX': (45.85, -108.61), 'KGGW': (48.21, -106.62), 'KTFX': (47.46, -111.39),
          'KMSX': (47.04, -113.99), 'KMHX': (34.78, -76.88), 'KRAX': (35.67, -78.49),
          'TCLT': (35.34, -80.88), 'TRDU': (36.0, -78.7), 'KLTX': (33.99, -78.43),
          'KBIS': (46.77, -100.76), 'KMVX': (47.53, -97.33), 'KMBX': (48.39, -100.86),
          'KUEX': (40.32, -98.44), 'KLNX': (41.96, -100.58), 'KOAX': (41.32, -96.37),
          'TEWR': (40.59, -74.27), 'KABX': (35.15, -106.82), 'KFDX': (34.63, -103.62),
          'KHDX': (33.08, -106.12), 'KLRX': (40.74, -116.8), 'KESX': (35.7, -114.89),
          'KRGX': (39.75, -119.46), 'TLAS': (36.14, -115.01), 'KENX': (42.59, -74.06),
          'KBGM': (42.2, -75.98), 'KBUF': (42.95, -78.74), 'KTYX': (43.76, -75.68),
          'TJFK': (40.59, -73.88), 'KOKX': (40.87, -72.86), 'KCLE': (41.41, -81.86),
          'TCVG': (38.9, -84.58), 'TLVE': (41.29, -82.01), 'TCMH': (40.01, -82.72),
          'TDAY': (40.02, -84.12), 'KILN': (39.42, -83.82), 'KFDR': (34.36, -98.98),
          'KTLX': (35.33, -97.28), 'TOKC': (35.28, -97.51), 'TTUL': (36.07, -95.83),
          'KINX': (36.18, -95.56), 'KVNX': (36.74, -98.13), 'KMAX': (42.08, -122.72),
          'KPDT': (45.69, -118.85), 'KRTX': (45.71, -122.96), 'KDIX': (39.95, -74.41),
          'KPBZ': (40.53, -80.22), 'KCCX': (40.92, -78.0), 'TPHL': (39.95, -75.07),
          'TPIT': (40.5, -80.49), 'TJUA': (18.12, -66.08), 'TSJU': (18.47, -66.18),
          'KCLX': (32.66, -81.04), 'KCAE': (33.95, -81.12), 'KGSP': (34.88, -82.22),
          'KABR': (45.46, -98.41), 'KUDX': (44.12, -102.83), 'KFSD': (43.59, -96.73),
          'KMRX': (36.17, -83.4), 'KNQA': (35.34, -89.87), 'KOHX': (36.25, -86.56),
          'TMEM': (34.9, -89.99), 'TBNA': (35.98, -86.66), 'KAMA': (35.23, -101.71),
          'KBRO': (25.92, -97.42), 'KGRK': (30.72, -97.38), 'KCRP': (27.78, -97.51),
          'KFWS': (32.57, -97.3), 'KDYX': (32.54, -99.25), 'KEPZ': (31.87, -106.7),
          'KHGX': (29.47, -95.08), 'KDFX': (29.27, -100.28), 'KLBB': (33.65, -101.81),
          'KMAF': (31.94, -102.19), 'KSJT': (31.37, -100.49), 'KEWX': (29.7, -98.03),
          'TDAL': (32.93, -96.97), 'TDFW': (33.06, -96.92), 'THOU': (29.52, -95.24),
          'TIAH': (30.06, -95.57), 'KICX': (37.59, -112.86), 'KMTX': (41.26, -112.45),
          'TSLC': (40.97, -111.93), 'KFCX': (37.02, -80.27), 'KLWX': (38.98, -77.49),
          'TIAD': (39.08, -77.53), 'KAKQ': (36.98, -77.01), 'KCXX': (44.51, -73.17),
          'KLGX': (47.12, -124.11), 'KATX': (48.19, -122.5), 'KOTX': (47.68, -117.63),
          'KGRB': (44.5, -88.11), 'KARX': (43.82, -91.19), 'KMKX': (42.97, -88.55),
          'TMKE': (42.82, -88.05), 'KRLX': (38.31, -81.72), 'KCYS': (41.15, -104.81),
          'KRIW': (43.07, -108.48)}

raobs = {'PANC': '70273', 'PANT': '70398', 'PABR': '70026',
         'PABE': '70219', 'PACD': '70316', 'PASY': '70414',
         'PAFA': '70261', 'PAKN': '70326', 'PADQ': '70350',
         'PAOT': '70133', 'PAMC': '70231', 'PAOM': '70200',
         'PASN': '70308', 'PAYA': '70361', 'KBMX': '72230',
         'KLZK': '72340', 'KFGZ': '72376', 'KTWC': '72274',
         'KEDW': '72381', 'KNKX': '72293', 'KOAK': '72493',
         'KVBG': '72393', 'KDNR': '72469', 'KGJT': '72476',
         'KJAX': '72206', 'KMFL': '72202', 'KTLH': '72214',
         'KTBW': '72210', 'KFFC': '72215', 'KBOI': '72681',
         'KDDC': '72451', 'KTOP': '72456', 'KLCH': '72240',
         'KSHV': '72248', 'KLIX': '72233', 'KCAR': '72712',
         'KDTX': '72632', 'KAPX': '72634', 'KMPX': '72649',
         'KINL': '72747', 'KSGF': '72440', 'KJAN': '72235',
         'KGGW': '72768', 'KTFX': '72776', 'KGSO': '72317',
         'KMHX': '72305', 'KBIS': '72764', 'KLBF': '72562',
         'KOAX': '72558', 'KABQ': '72365', 'KEPZ': '72364',
         'KLKN': '72582', 'KVEF': '72388', 'KREV': '72489',
         'KALB': '72518', 'KOKX': '72501', 'KBUF': '72528',
         'KILN': '72426', 'KOUN': '72357', 'KMFR': '72597',
         'KSLE': '72694', 'KPIT': '72520', 'KCHS': '72208',
         'KABR': '72659', 'KUNR': '72662', 'KBNA': '72327',
         'KAMA': '72363', 'KBRO': '72250', 'KCRP': '72251',
         'KDRT': '72261', 'KFWD': '72249', 'KMAF': '72265',
         'KSLC': '72572', 'KRNK': '72318', 'KWAL': '72402',
         'KIAD': '72403', 'KUIL': '72797', 'KOTX': '72786',
         'KGRB': '72645', 'KRIW': '72672'}

In [ ]:
# @title <font size="+3" color="green">Make date/time/radar selections</font>

sounding_type = "model" # @param ["model","raob"]

# @markdown You may just want to use the latest hour...
use_latest_time = False # @param {"type":"boolean"}

# @markdown If not using current time, make your selections below:
UTC_date = "2024-05-07" # @param {type:"date"}
# @markdown If retrieving a raob, be sure to choose 00 or 12 UTC below:
UTC_hour = "22" # @param ["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17,","18","19","20","21","22","23"]

# @markdown Select your radar:
radar = "KGRR" # @param {"type":"string","placeholder":"KGRR"}

try:
  lat, lon = radars[radar]
  print(f"Radar: {radar}")
  print(f"Latitude: {lat}")
  print(f"Longitude: {lon}")
except KeyError:
  print("*** INVALID RADAR ***")
  print("Enter a valid radar and run this cell again.")

if sounding_type == "raob":
  try:
    id = raobs[radar]
    print(f"Station ID: {id}")
  except KeyError:
    print("*** RAOB NOT AVAILABLE FOR THIS RADAR ***")
    print("Changing to model sounding...")
    sounding_type = "model"


Radar: KGRR
Latitude: 42.89
Longitude: -85.54


In [ ]:
# @title <font size="+3" color="green">Run this cell to get your model or raob file</font>

model_base_url = 'https://mesonet.agron.iastate.edu/api/1/nws/bufkit.json'
raob_base_url = 'https://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST'

ending = """
"units": {"pressure": ["MB", "millibars", "hPa", "hectopascals"],
"height": ["M", "meters"],
"temperature": ["C", "celsius"],
"dewpoint": ["C", "celsius"],
"wind_from": ["DEG", "degrees"],
"wind_speed": ["MPS", "meters per second"],
"uvv": ["UBS", "microbars per second"]}
}
"""

#regex = re.findall(TIME_REGEX, line)
#new_line = line.replace(f"{regex[0]} {regex[1]}",
#f"{new_datestring_1} {new_datestring_2}")

def extract_datetime(date_str):
    year = date_str[:4]
    month = date_str[5:7]
    day = date_str[8:10]
    return year, month, day


def make_dummy_raob_header(dummy_time):
    """
    This is just to conform to the format required of GR2A.
    """
    top = {}
    top["time"] = f"{UTC_date}T{UTC_hour}:01:00Z"
    top["time"] = dummy_time
    top["lat"] = 42.88
    top["lon"] = -85.52
    source_dict = {}
    source_dict["type"] = "model"
    source_dict["model"] = "RAP"
    source_dict["run_time"] = dummy_time
    source_dict["forecast_hour"] = 0
    source_json = source_dict
    top["source"] = source_json
    starting = json.dumps(top)
    return str(starting)[:-1]

def write_raob_data(fixed_list) -> None:
    starting = make_dummy_raob_header()
    with open('sounding_data.txt', 'w') as fout:
        fout.write(f'{starting},')
        fout.write(f'"levels": {fixed_list},')
        fout.write(ending)

def request_raob_data(url):
    """
    Requests data from the University of Wyoming site and
    parses it into a JSON object.
    """
    # Create a session with retries disabled
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=0)
    session.mount('https://', adapter)
    session.mount('http://', adapter)

    # Fetch the data
    response = session.get(url, verify=False, timeout=10)
    data = response.text
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(data, 'html.parser')

    # Extract the <PRE> content and parse out lines
    try:
        pre_content = soup.find('pre').text
        lines = pre_content.split('\n')
    except AttributeError:
      return "*** NO DATA AVAILABLE ***"
    # Extract the relevant data
    levels_list = []
    for line in lines:
        if line.strip() and not line.startswith(('----')):
            parts = line.split()
            test = parts[0]
            if test[:1].isdigit() and len(test) >=5 and len(parts) >= 4:
                level = {}
                level["pressure"] = float(parts[0])
                level["height"] = float(parts[1])
                level["temperature"] = float(parts[2])
                levels_list.append(level)
    fixed_levels_list = json.dumps(levels_list)
    return fixed_levels_list

def request_model_sounding(url):
    """
    Requests data from the Iowa State site
    """
    # Create a session with retries disabled
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=0)
    session.mount('https://', adapter)
    session.mount('http://', adapter)

    # Fetch the data
    TIME_REGEX = "[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}Z"
    try:
        response = session.get(url, verify=False, timeout=10)
        data = response.text

        regex = re.findall(TIME_REGEX, data)
        fixed = regex[0][:-5] + '1:00Z'
        new_text = data.replace(regex[0], fixed)
        with open('model_sounding.txt', 'w', encoding='utf-8') as fout:
            fout.write(new_text)
        return "model_sounding.txt file created successfully"
    except:
        return "Sounding creation failed!"


def create_raob_url(year, month, day, hour, id) -> str:
    """
    Formats url for U of WY website. Example:
    https://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2024&MONTH=05&FROM=0800&TO=0800&STNM=72632
    """
    dh = f"{day}{hour}"
    url = f"{raob_base_url}&YEAR={year}&MONTH={month}&FROM={dh}&TO={dh}&STNM={id}"
    print(f"Raob URL used: \n{url}\n")
    return url

def get_model_sounding_url(UTC_date, UTC_hour, lat, lon):
    """
    Retrieves model from ISU json Bufkit API:
    https://mesonet.agron.iastate.edu/api/1/nws/bufkit.json
    """
    hour_str = f"{UTC_hour}:02:00"
    address = f"{model_base_url}?time={UTC_date}T{hour_str}&lon={lon}&lat={lat}&gr=1"
    return address


###### Code below is to obtain most recent or historical model or raob data

year, month, day = extract_datetime(UTC_date)
if sounding_type == "model":
    if use_latest_time:
        now = datetime.now()
        UTC_date = now.strftime("%Y-%m-%d")
        UTC_hour = now.strftime("%H")
    url = get_model_sounding_url(UTC_date, UTC_hour, radars[radar][0], radars[radar][1])
    print(f"Model sounding URL used: \n{url}\n")
    print(request_model_sounding(url))

if sounding_type == "raob":
    if use_latest_time:
        now = datetime.now()
        UTC_date = now.strftime("%Y-%m-%d")
        UTC_hour = now.strftime("%H")
        mod = int(UTC_hour)%12
        print(mod)
        if mod > 2:
            hour_delta = mod
        else:
            hour_delta = 12 + mod
        fixed = now - timedelta(hours=hour_delta, minutes=5)
        year, month, day = extract_datetime(fixed.strftime("%Y-%m-%d"))
        UTC_hour = fixed.strftime("%H")
        raob_header_time = fixed.strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        year, month, day = extract_datetime(UTC_date)
        raob_header_time = f"{UTC_date}T{UTC_hour}:01:00Z"
    start = make_dummy_raob_header(raob_header_time)
    url = create_raob_url(year, month, day, UTC_hour, id)
    fixed_levels = request_raob_data(url)
    if fixed_levels == "*** NO DATA AVAILABLE ***":
        print("CANNOT OBTAIN RAOB DATA")
    else:
        with open('sounding_data.txt', 'w') as fout:
            fout.write(f'{start},\n')
            fout.write(f'"levels": \n')
            fout.write(f"{fixed_levels},")
            fout.write(ending)
            print("'sounding_data.txt' ready for download")

Model sounding URL used: 
https://mesonet.agron.iastate.edu/api/1/nws/bufkit.json?time=2024-05-07T22:02:00&lon=-85.54&lat=42.89&gr=1

model_sounding.txt file created successfully


In [ ]:
# @title <font size="+3" color="green">Ignore this cell</font>
# for f in range(0,50000,1000):
#   h = f / 3.28084
#   p = 1013.25 * (1 - 2.25577 * 10**(-5)* h)**5.25588

#   print(round(p),f,int(round(h)))


test = 'asads 2024-12-24T12:00:00Z aasdsda'
import re
TIME_REGEX = "[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}Z"
regex = re.findall(TIME_REGEX, test)
fixed = regex[0][:-5] + '2:00Z'
new_test = test.replace(regex[0], fixed)
print(new_test)

asads 2024-12-24T12:02:00Z aasdsda
